In [1]:
import tensorflow   # machine learning library
import stellargraph   # tools for graphs
# import networkx   # more tools for graphs
import numpy   # scientific computation and variable formats library
import matplotlib   # all plotting except graphs
import plotly.graph_objects as plotly   # plotting graphs
import os.path   # interaction with  files and filenames
import pandas   # library for loading and organising data
import datetime   # calendar and dates handling
import re   # I'm using regex, apologies

Please notice, to open jupyter in custom directory use:
>jupyter notebook --notebook-dir=[your_directory]

Expecting file hierarchy:
    <ul>root/
        <li>Antarctica_research_2020/
            <ul>.git/</ul>
            <ul>.gitignore</ul>
            <ul>readme.md</ul>
            <ul>notebook.ipynb</ul></li>
        <li>data/
            <ul>Sesja 1.xlsx</ul>
            <ul>.</ul>
            <ul>.</ul>
            <ul>Sesja 26.xlsx</ul></li>
    </ul>

In [2]:
# Let me write in advantage examined peoples' occupations:
ppl_professions = {1: "fisherman",
                  2: "cook",
                  3: "medic",
                  4: "deputy technical manager",
                  5: "carpenter",
                  6: "driver",   # UWAGA - nr 6 przybył na stację 25.04.79r
                  7: "technician",
                  8: "chemist",
                  9: "meteorologist",
                  10: "ichthyologist",   # study of fish
                  11: "magnetologist",
                  12: "seismologist",
                  13: "biologist",
                  14: "electrician",
                  15: "radiotelegraphist",
                  16: "psychologist",
                  17: "electronics engineer",
                  18: "meteorologist",
                  19: "microbiologist",
                  20: "parasitologist",
                  21: "botanist"
                  }

In [3]:
data_path = "../data/"
file_list = [name for name in os.listdir(data_path)]
print(file_list)

['Sesja 1.xlsx', 'Sesja 10.xlsx', 'Sesja 11.xlsx', 'Sesja 12.xlsx', 'Sesja 13.xlsx', 'Sesja 14.xlsx', 'Sesja 15.xlsx', 'Sesja 16.xlsx', 'Sesja 17.xlsx', 'Sesja 18.xlsx', 'Sesja 19.xlsx', 'Sesja 2.xlsx', 'Sesja 20.xlsx', 'Sesja 21.xlsx', 'Sesja 22.xlsx', 'Sesja 23.xlsx', 'Sesja 24.xlsx', 'Sesja 25.xlsx', 'Sesja 26.xlsx', 'Sesja 3.xlsx', 'Sesja 4.xlsx', 'Sesja 5.xlsx', 'Sesja 6.xlsx', 'Sesja 7.xlsx', 'Sesja 8.xlsx', 'Sesja 9.xlsx']


In [4]:
# Sessions order is important, let me get sorted file info
sorted_filenames = dict()
for name in file_list:
    list_of_digits = list(map(int, filter(str.isdigit, name)))
    sorted_filenames[name] = list_of_digits[0] if len(list_of_digits) == 1 else list_of_digits[1] + list_of_digits[0]*10
sorted_filenames = sorted(sorted_filenames.items(), key=lambda el: el[1])
sorted_filenames = {val: key for key, val in sorted_filenames}
print(sorted_filenames)

{1: 'Sesja 1.xlsx', 2: 'Sesja 2.xlsx', 3: 'Sesja 3.xlsx', 4: 'Sesja 4.xlsx', 5: 'Sesja 5.xlsx', 6: 'Sesja 6.xlsx', 7: 'Sesja 7.xlsx', 8: 'Sesja 8.xlsx', 9: 'Sesja 9.xlsx', 10: 'Sesja 10.xlsx', 11: 'Sesja 11.xlsx', 12: 'Sesja 12.xlsx', 13: 'Sesja 13.xlsx', 14: 'Sesja 14.xlsx', 15: 'Sesja 15.xlsx', 16: 'Sesja 16.xlsx', 17: 'Sesja 17.xlsx', 18: 'Sesja 18.xlsx', 19: 'Sesja 19.xlsx', 20: 'Sesja 20.xlsx', 21: 'Sesja 21.xlsx', 22: 'Sesja 22.xlsx', 23: 'Sesja 23.xlsx', 24: 'Sesja 24.xlsx', 25: 'Sesja 25.xlsx', 26: 'Sesja 26.xlsx'}


In [5]:
# Loading data
data_dict = dict()
for key, val in sorted_filenames.items():
    data_dict[key] = pandas.read_excel(data_path + val, header=None, dtype=object)

In [6]:
rand_int = int(numpy.random.rand() * 26)
data_dict[rand_int].head(4)
# An example:

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,NaN,Pozycja osoby,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,N = 21,NaN,NaN,1979-10-21 00:00:00,NaN,NaN,NaN,ZIM22,NaN,NaN
1,Nr O.B.,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
2,1,2717,3014,3023,2323,2312,2512,2318,2623,3120,...,2415,2422,3010,2710,2521,2721,3215,2115,2119,2320
3,2,2312,2201,2016,3418,2124,2627,2914,3407,1622,...,1835,1616,0910,2007,1511,2823,2432,1830,1218,0827


The target format for data is a stellargraph graph for each session (session -> data recording occuring every two week). But before that, let me read the data into intermediate objects: Session(s) and Person(s).
    Session will hold data in respect to each source data file, organising entries into lists of coordinates marked by examined people - one list for one man for each session.
    Person will organise data in respect to a single individual. Each* individual hold data for all 26 sessions, for convienience repeated in several arrangements.
    
*except mr. 6th, since he arrived later

In [7]:
class Session(object):
    def __init__(self, raw_session_data):
        self.date = self.find_date(raw_session_data)
        self.relations = self.get_relations(raw_session_data) # dictionary of lists of tuples - column_number : [(x1, y1), (x2, y2), ...]
    
    def get_relations(self, raw_session_data) -> dict:
        rels = dict()
        col_no = len(ppl_professions)
        rels = {x: [] for x in range(1, col_no + 1)}
        for person in range(2, col_no + 1):
            for entry in range(1, col_no):
                string_value = str(raw_session_data.iloc[person, entry])
                subject_no = raw_session_data.iloc[person, 0]
                data_point = self.tuple_from_string(string_value)
                rels[subject_no].append(data_point)
                
        return rels
    
    def tuple_from_string(self, string) -> tuple:
        if string == 'nan':
            return None
        elif len(string) == 4:
            x = int(string[1]) + int(string[0]) * 10
            y = int(string[3]) + int(string[2]) * 10
            return (x, y)
        elif len(string) == 3:
            x = int(string[0])
            y = int(string[2]) + int(string[1]) * 10
            return (x, y)
        else:
            return None
    
    def find_date(self, raw_session_data) -> str:
        for key, col in raw_session_data.iteritems():
            for val in col:
                regex = re.search(r"\d{1,2}.\d{1,2}.\d{4}|\d{4}.\d{1,2}.\d{1,2}", str(val))
                if regex:
                    return regex.group()
                
    def __str__(self):
        random = int(numpy.random.rand() * len(ppl_professions))
        return "Session from " + self.date + " example entry: " + str(self.relations[random]) + " for " + str(random)

In [77]:
session_data = [Session(e) for e in data_dict.values()]
# An example entry:
print(session_data[-1])

Session from 1980-01-10 example entry: [(35, 7), (39, 7), (41, 1), (6, 32), (29, 11), (18, 15), (9, 26), (27, 22), (20, 12), (39, 3), (40, 10), (13, 26), (18, 19), (33, 13), (19, 23), (31, 14), (23, 13), (23, 25), (12, 30), (23, 19)] for 20


In [126]:
class Person(object):
    def __init__(self, person_no, particular_person_profession, session_data):
        self.person_no = person_no
        self.profession = particular_person_profession
        self.sessions_dict = self.get_all_relations(session_data)
        self.relations_vector = self.get_all_relative_vectors(self.sessions_dict)
        self.distances = self.get_all_distances(self.relations_vector)
        
    def get_all_relations(self, session_data) -> dict:
        out_dict = {x: [] for x in sorted_filenames.keys()}
        for no in out_dict.keys():
            out_dict[no] = session_data[no - 1].relations[self.person_no]
        return out_dict
    
    def get_all_relative_vectors(self, relations) -> dict:
        out_dict = {x: [] for x in sorted_filenames.keys()}
        for no, session in relations.items():
            for relation in session:
                if relation is not None and session[self.person_no - 1] is not None:
                    out_dict[no].append((relation[0] - session[self.person_no - 1][0], relation[1] - session[self.person_no - 1][1]))
                else:
                    out_dict[no].append(None)
        return out_dict
    
    def get_all_distances(self, vectors) -> dict:
        out_dict = {x: [] for x in sorted_filenames.keys()}
        for no, session in vectors.items():
            for vector in session:
                if vector is not None:
                    out_dict[no].append(self.calc_distance(vector[0], vector[1]))
                else:
                    out_dict[no].append(None)
        return out_dict
    
    def get_particular_rel_list(self, other) -> list:
        pass
    
    def calc_distance(self, a, b) -> float:
        return numpy.sqrt(a ** 2 + b ** 2)
    
    def __str__(self):
        rand_session = int(numpy.random.rand() * 26)
        return "No. " + str(self.person_no) + " being " + self.profession + " with distances: " + str(self.distances[rand_session]) + " for session " + str(rand_session)

In [127]:
people_data = [Person(no, profession, session_data) for no, profession in ppl_professions.items()]
# An example:
rand_ppl = int(numpy.random.rand() * 21)
print(people_data[rand_ppl])

No. 1 being fisherman with distances: [0.0, 2.23606797749979, 8.246211251235321, 8.06225774829855, 9.486832980505138, 7.211102550927978, 12.041594578792296, 4.123105625617661, 11.40175425099138, 2.0, 7.0, 12.206555615733702, 6.4031242374328485, 10.770329614269007, 2.8284271247461903, 2.0, 8.246211251235321, 7.810249675906654, 14.212670403551895, 10.198039027185569] for session 5
